In [2]:
# Dependencies
import requests
import json
import pandas as pd
from pprint import pprint
from config import client_id, client_secret

Following instructions from: https://stmorse.github.io/journal/spotify-api.html
AND from Spotify documentation:
https://developer.spotify.com/documentation/general/guides/authorization-guide/#list-of-scopes

## Authorization flow

Step 1: Request authorization to access data. No code. 
- Example Grant access URL:
https://accounts.spotify.com/authorize?response_type=code&client_id=5d67ef47a63a42f08feb4c824ee174f7&scope=user-library-read&redirect_uri=https%3A%2F%2Fgoogle.com%2Fcallback&show_dialog=true

Once accept, data can now be requested.

Note: if changing `scope`, this step and step 2 need to be repeated.
Scope for the following:
- User's Top Artists and Tracks: `user-top-read`
- List of a User's Playlists: `playlist-read-private`

In [3]:
# Once access is granted, copy the code from the URL
code = 'AQDHk5w01qWgoh8ClHrMHg9arLydL0zy6Zs2l3bl3RP0kEYfdHViimQ1A0aWOwrA30WuMlwoW39eUCPkjhaUwNvjZwwMOtIknkpUnf58oXQxg4twZ-xAahJfdCIqKDTbUVwsdxD4PTprqQ3Jdnv-iLlEgPrhQhRMucCOWVpCE2aRSVOJDWtwuF69ZebIXw'

Step 2: Request to get refresh and access token
- Make POST request to Spotify on /api/token endpoint
- Require parameters specfied accordingly.

In [4]:
# POST - REQUEST BODY PARAMETER, skipping header parameter using alternative way
request_body_parameter = {
    'grant_type': 'authorization_code',
    'code': code,
    'redirect_uri': 'https://google.com/callback',
    'client_id': client_id,
    'client_secret': client_secret,
}

In [5]:
# Pass an access token - POST a request with client credentials and save part of response
AUTH_URL = 'https://accounts.spotify.com/api/token'

# POST - REQUEST BODY PARAMETER
auth_response = requests.post(AUTH_URL, request_body_parameter)

# convert the response to JSON
auth_response_data = auth_response.json()

# Try print it out to get refresh_token
print(auth_response_data)

{'access_token': 'BQDiFXFPKMyXpLcZJAab7crMEaGXRRA4eOj4iyDcpKaTudLEFIiBI47sfgPik6GRw9pe5jjbhQJcCKhzAhl7BvWVKGf_Tjz-S502zsVis-CXDJGU7-_E7209s_Iuo-aHLUZLPymq3WmJ22J0oiQ', 'token_type': 'Bearer', 'expires_in': 3600, 'refresh_token': 'AQA1YePrNTTY_I-kOBiWb-fk9Wm6F-mOjvfSTm25jjVvfgqtkkQVs-HESP_zRW802itP0DKSqIOhTA43yj9Y66fpM86y6nmfaAgqFKoq31kc7p5ftqF8Md6PiVJcmTQD-To', 'scope': 'user-top-read'}


In [6]:
# Save the access token
access_token = auth_response_data['access_token']

# Save the refresh token
refresh_token = auth_response_data['refresh_token']

Step 3: Use access token to access data of interests(?)
- Note: when access token expires, it can't be used anymore.

#### Get a User's Top Artists and Tracks

In [162]:
# Form GET request to API server with access_token in the header
headers = {
    'Authorization': f'Bearer {access_token}'
}

# base URL of all Spotify API endpoints
base_url = 'https://api.spotify.com/v1/'

# user's top track
top_track = 'me/top/tracks'
time_range = 'time_range=medium_term'
limit = 'limit=50'
offset = 'offset=0'

### If token expires, go to Step 4 and run those code and run the code below

In [163]:
# Perfrom a request for data
track_json = requests.get(base_url + top_track + '?' + time_range + '&' + limit + '&' + offset
                          , headers=headers).json()

In [164]:
# Save outout to text file to go parse later...
with open('top_tracks.json', 'w') as outfile:
    json.dump(track_json, outfile, indent=2)

In [165]:
# Write a function that uses map to take the data in array (either array of array or array of object, etc) 
## and apply it in each item in an array - Return me an array from an array with only the pieces I want

# Instead of define a bunch of empty lists and create a loop to append each item in to each list, this is much cleaner and more efficient

def track_func(track):
#artists = [artist['name'] for artist in track['artists']]
# artist['name']
# { 'id': artist['id'], 'name': artist['name'] }
    artists_id = [ { 'id': artist['id'] } for artist in track['artists'] ]
    artists_name = [ { 'name': artist['name'] } for artist in track['artists'] ]
    album_id = track['album']['id']
    album_name = track['album']['name']
    release_date = track['album']['release_date']
    song_id = track['id']
    title = track['name']
    popularity = track['popularity']
    duration_ms = track['duration_ms']
    song_url = track['external_urls']
    
    return {
        'artists_id': artists_id,
        'artists_name': artists_name,
        'album_id': album_id,
        'album_name':album_name,
        'release_date':release_date,
        'song_id': song_id,
        'title': title,
        'popularity': popularity,
        'duration_ms': duration_ms,
        'song_url': song_url
        
    }
# Use map to give a list iterator
track_list_iterator = map(track_func, track_json['items'])
# Turn iterator into a list
track_list = list(track_list_iterator)
track_list

[{'artists_id': [{'id': '64tJ2EAv1R6UaZqc4iOCyj'}],
  'artists_name': [{'name': 'YOASOBI'}],
  'album_id': '41HUxKwnbrg8IdelmMibj9',
  'album_name': '怪物',
  'release_date': '2021-01-06',
  'song_id': '06XQvnJb53SUYmlWIhUXUi',
  'title': '怪物',
  'popularity': 82,
  'duration_ms': 206000,
  'song_url': {'spotify': 'https://open.spotify.com/track/06XQvnJb53SUYmlWIhUXUi'}},
 {'artists_id': [{'id': '6Na6YdJLEeBDbP1bZv6q7a'},
   {'id': '2EZcXq4boUMbuGRsSWbjeC'}],
  'artists_name': [{'name': '国江田計 (CV: 阿部敦)'}, {'name': '都築潮 (CV: 川原慶久)'}],
  'album_id': '7gU3VRaRiSBPaynGeLjhTJ',
  'album_name': '世界とかくれんぼ',
  'release_date': '2020-12-04',
  'song_id': '6vc3GcAg27Z9iknbjBVZtH',
  'title': '世界とかくれんぼ',
  'popularity': 22,
  'duration_ms': 234888,
  'song_url': {'spotify': 'https://open.spotify.com/track/6vc3GcAg27Z9iknbjBVZtH'}},
 {'artists_id': [{'id': '449AEgfeOxqAuRn0uX6l3u'}],
  'artists_name': [{'name': 'UNISON SQUARE GARDEN'}],
  'album_id': '6Rm9rTuxsvukGVowLHP93c',
  'album_name': 'UNISON 

In [166]:
top_tracks_df = pd.DataFrame(track_list)

In [168]:
top_tracks_df

,artists_id,artists_name,album_id,album_name,release_date,song_id,title,popularity,duration_ms,song_url
0,[{'id': '64tJ2EAv1R6UaZqc4iOCyj'}],[{'name': 'YOASOBI'}],41HUxKwnbrg8IdelmMibj9,怪物,2021-01-06,06XQvnJb53SUYmlWIhUXUi,怪物,82,206000,{'spotify': 'https://open.spotify.com/track/06...
1,"[{'id': '6Na6YdJLEeBDbP1bZv6q7a'}, {'id': '2EZ...","[{'name': '国江田計 (CV: 阿部敦)'}, {'name': '都築潮 (CV...",7gU3VRaRiSBPaynGeLjhTJ,世界とかくれんぼ,2020-12-04,6vc3GcAg27Z9iknbjBVZtH,世界とかくれんぼ,22,234888,{'spotify': 'https://open.spotify.com/track/6v...
2,[{'id': '449AEgfeOxqAuRn0uX6l3u'}],[{'name': 'UNISON SQUARE GARDEN'}],6Rm9rTuxsvukGVowLHP93c,UNISON SQUARE GARDEN The 1st Collection for st...,2019-10-11,4kFY1RxOo6Wd8hgVL3t6Rb,Phantom Joke,52,244440,{'spotify': 'https://open.spotify.com/track/4k...
3,[{'id': '1xwuTSJVlnBEOecAEHGRfY'}],[{'name': 'ITOWOKASHI'}],178MyVBgrjKDFvwYclbee6,中央突破,2017-06-21,4EPXOp2cwHlYiOpz2YQWYS,スターダスト,27,271360,{'spotify': 'https://open.spotify.com/track/4E...
4,[{'id': '3i7tuzRfutJGAUowspOM1f'}],[{'name': 'Kashitaro Ito'}],3ewKbf1sUhzYyGSn8LkpHe,ENDING THEME FROM TV SERIES ”DECADENCE”,2020-07-29,3REf7M4gfHUaS8SAJrrLG7,ARK OF MEMORIES,34,252100,{'spotify': 'https://open.spotify.com/track/3R...
5,[{'id': '6mEQK9m2krja6X1cfsAjfl'}],[{'name': 'Ado'}],7hy4zddDO6zOMQxRipqQry,レディメイド,2020-12-24,6IP3GvRj6Rc65rly1rIHq5,レディメイド,65,244377,{'spotify': 'https://open.spotify.com/track/6I...
6,[{'id': '3i7tuzRfutJGAUowspOM1f'}],[{'name': 'Kashitaro Ito'}],5VfxZpps2s3yibA445smqp,一意専心,2014-01-22,62GRSSM4SdDSp5MXgWdnah,Calc.,37,226184,{'spotify': 'https://open.spotify.com/track/62...
7,[{'id': '6mEQK9m2krja6X1cfsAjfl'}],[{'name': 'Ado'}],5msh7Lspyh1jGbDM2BxwAy,うっせぇわ,2020-10-23,6EzZn96uOc9JsVGNRpx06n,うっせぇわ,79,204920,{'spotify': 'https://open.spotify.com/track/6E...
8,[{'id': '3i7tuzRfutJGAUowspOM1f'}],[{'name': 'Kashitaro Ito'}],5VfxZpps2s3yibA445smqp,一意専心,2014-01-22,3bnSwZS7PJOCjqZF38iBdZ,rebirthday,18,313470,{'spotify': 'https://open.spotify.com/track/3b...
9,[{'id': '1xwuTSJVlnBEOecAEHGRfY'}],[{'name': 'ITOWOKASHI'}],178MyVBgrjKDFvwYclbee6,中央突破,2017-06-21,5meZvaHT6pQx4AZP1f8lN3,カナデアイ,48,233173,{'spotify': 'https://open.spotify.com/track/5m...


Step 4: When access token expires, refresh token to the rescue (from step 2)

In [150]:
# Base 64 encoded string that contains the client ID and client secret key.
import base64

def encode_base64(message):
    message_bytes = message.encode('ascii')
    base64_bytes = base64.b64encode(message_bytes)
    base64_message = base64_bytes.decode('ascii') 
    return base64_message
#     return str(base64_bytes, "utf-8")
    
print(encode_base64(f'{client_id}:{client_secret}'))

NWQ2N2VmNDdhNjNhNDJmMDhmZWI0YzgyNGVlMTc0Zjc6Y2I0YzcyMjZhNjcyNDMzNjg0MzI2MGQ5ZWYyZTdiNjk=


In [151]:
# Get the new refresh access token and update - only when request fails 
def refresh_access_token():   
    # build body parameter
    body = {
        'grant_type': 'refresh_token',
        'refresh_token': refresh_token
    }
    
    # base64 string from above encode_base64 function
    auth_base64 = encode_base64(f'{client_id}:{client_secret}')
    
    # build header parameter
    header = {
        'Authorization': f'Basic {auth_base64}'
    }
    
    request_access_token = requests.post(AUTH_URL, body, headers=header)
    request_access_token_data = request_access_token.json()
    print(request_access_token_data)
    # Update access_token (for step 3)
    return request_access_token_data['access_token']

In [152]:
# Update access token from Step 3
access_token = refresh_access_token()

# Update headers
headers = {'Authorization': f'Bearer {access_token}'}

{'access_token': 'BQANFuZEWfYCaRARgufCSefzw4XPhuFHpaWBm9ePYBuT6CA4zgAQdMTEsFlL6N1NRUPMkaS1QRhrWRGSmFm592JtIj2LsaVdok97a-QtQRBqNlls8iiYYcfPHrvDuTZ6aJnD2hlgft9tlK6UT3_vgckqT9vFeA', 'token_type': 'Bearer', 'expires_in': 3600, 'scope': 'user-library-read user-top-read'}


# Future Update
- Save the current time + `expires_in` into a variable, `expire_time`.
- Evey time you make a request for data, add a line of code that checks "Is my current time > `expire_time`?"
- If yes, refresh access token and update headers (step 4)